#### HuggingFace pre-trained model link
[Bert](https://huggingface.co/google-bert/bert-base-uncased)
#### HugginFace Dataset link
[Imdb](https://huggingface.co/datasets/stanfordnlp/imdb)

### Importing warnings

In [ ]:
import warnings
warnings.filterwarnings('ignore')

### Installing libraries

In [ ]:
! pip install datasets transformers torch huggingface_hub

### Selecting model and dataset from huggingface

In [ ]:
data="stanfordnlp/imdb"
model="google-bert/bert-base-uncased"

### Loading dataset from huggingface

In [ ]:
from datasets import load_dataset

In [ ]:
dataset=load_dataset(data)

In [ ]:
dataset

### Importing Tokenizer and model from huggingface

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer=AutoTokenizer.from_pretrained(model)

In [ ]:
## Tokeninzing datasets
def tokenize(sample):
    return tokenizer(sample['text'],padding='max_length',truncation=True)

tokenize_dataset=dataset.map(tokenize,batched=True)

In [ ]:
from transformers import AutoModelForSequenceClassification
bert_model=AutoModelForSequenceClassification.from_pretrained(model,num_labels=2)

### Selecting Params

In [ ]:
from transformers import Trainer,TrainingArguments
args=TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=1,
    weight_decay=0.01,
)
trainer=Trainer(
    model=bert_model,
    args=args,
    train_dataset=tokenize_dataset['train'],
    eval_dataset=tokenize_dataset['test'],
)

### Model fine-tuning

In [ ]:
trainer.train()

### Saving model for future usecase

In [ ]:
trainer.save_model("./result")

In [ ]:
tokenizer.save_pretrained("./result")

### Making zip file to download locally

In [ ]:
!zip -r my_model.zip ./result

In [ ]:
from google.colab import files
files.download("my_model.zip")

### Making inferencing from save model directory 

In [ ]:
fine_model=AutoModelForSequenceClassification.from_pretrained("./result")
fine_token=AutoTokenizer.from_pretrained("./result")

In [ ]:
fine_model.eval()

In [ ]:
input_text="I recently purchased amazon, and I must say that my experience has been quite disappointing. Initially, I had high hopes based on the marketing claims and positive reviews, but unfortunately, the reality did not live up to those expectations."
tokenize_text=fine_token(input_text,padding=True,truncation=True,return_tensors='pt')
with torch.no_grad():
    output=fine_model(**tokenize_text)
predictions = torch.argmax(output.logits, dim=-1)
print(f"Predicted label: {predictions.item()}")

### pushing model to huggingface (Make sure to use hf-token with write permission)

In [ ]:
from huggingface_hub import login
login()

### pushing model from saved directory to huggingface

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load your fine-tuned model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained("./result")
tokenizer = AutoTokenizer.from_pretrained("./result")

# Push to Hub
model.push_to_hub("kgpian/bert-sentiment-imdb")
tokenizer.push_to_hub("kgpian/bert-sentiment-imdb")

### huggingface fine-tune model [link](https://huggingface.co/kgpian/bert-sentiment-imdb)

### Making inferencing from hugginface fine-tuned model

In [ ]:
hf_model=AutoModelForSequenceClassification.from_pretrained("kgpian/bert-sentiment-imdb")
hf_tokenizer=AutoTokenizer.from_pretrained("kgpian/bert-sentiment-imdb")

In [ ]:
## taking input from user
user_input_text=input("Enter your review: ")
user_tokenize_input=hf_tokenizer(user_input_text,padding=True,truncation=True,return_tensors='pt')
with torch.no_grad():
    output=fine_model(**user_tokenize_input)
predictions = torch.argmax(output.logits, dim=-1)
print(f"Predicted label: {predictions.item()}")